# Определение пола по имени
Задание к занятию «Языковые модели. Счетные языковые модели и вероятностные языковые модели»

In [199]:
import os
import itertools
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [200]:
data_dir = "./data"
random_state = 777

## Определяем вспомогательные функции

In [201]:
def get_names_from_file(name):
    """
    Читает имена из файла
    :param str name: имя фацла (без расширения)
    :rtype: set
    """
    file_name = os.path.join(data_dir, name + '.txt')
    with open(file_name, 'r') as file:
        return set(map(str.strip, file.readlines()))
    
    
def make_names_dataset(names_male, names_female):
    """
    Создаёт pandas DataFrame из списков мужских и женских имён
    Колонки: name (индекс), gender (F/M категориальная)
    :rtype: pandas.DataFrame
    """    
    names = pd.DataFrame(
        sorted(itertools.chain(
            zip(names_female, itertools.repeat('F')),
            zip(names_male, itertools.repeat('M')),
        )),
        columns=['name', 'gender'],
    ).set_index('name')
    names['gender'] = names.gender.astype('category')
    return names
    
    
def load_and_prepare_names():
    """
    Загружает мужские и женские имена из фалов, 
    удаляет дубликаты и возвращает в виде pandas.DataFrame (name, gender)
    :rtype: pandas.DataFrame
    """
    print('Заружаем имена...')
    names_male = get_names_from_file('male')
    names_female = get_names_from_file('female')
    print("Мужcких имён: {}".format(len(names_male)))    
    print("Женских имён: {}".format(len(names_female)))

    # Ищем неоднозначные имена
    names_intersect = names_male.intersection(names_female)
    print("\nНеоднозначные имена: {}... ({})".format(
        ', '.join(sorted(names_intersect)[:7]),
        len(names_intersect)
    ))

    print('\nУбираем неоднозначные имена...')
    names_male -= names_intersect
    names_female -= names_intersect
    print("Мужcких имён: {}".format(len(names_male)))    
    print("Женских имён: {}".format(len(names_female)))
    
    return make_names_dataset(names_male, names_female)

In [202]:
def split_names_to_train_test(names):
    """
    Разбивает датасет имён на тестовую и обучающие выборки
    """
    X, y = names.index, names.gender.cat.codes
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=random_state)
    print("Размер обучающей выбрки: {}".format(X_train.shape[0]))
    print("Размер тестовой выбрки: {}".format(X_test.shape[0]))
    print("Процент мужчин в обучающей выборке: {:.0f}%".format(sum(y_train) / len(y_train) * 100))
    print("Процент мужчин в тестовой выборке: {:.0f}%".format(sum(y_test) / len(y_test) * 100))    
    return X_train, X_test, y_train, y_test 

## Загружаем данные

In [203]:
names = load_and_prepare_names()
names.head()

Заружаем имена...
Мужcких имён: 2943
Женских имён: 5000

Неоднозначные имена: Abbey, Abbie, Abby, Addie, Adrian, Adrien, Ajay... (365)

Убираем неоднозначные имена...
Мужcких имён: 2578
Женских имён: 4635


,gender
name,
Aamir,M
Aaron,M
Abagael,F
Abagail,F
Abbe,F


## Делим на обучающую и тестовую выборки

In [204]:
X_train, X_test, y_train, y_test = split_names_to_train_test(names)

Размер обучающей выбрки: 5409
Размер тестовой выбрки: 1804
Процент мужчин в обучающей выборке: 36%
Процент мужчин в тестовой выборке: 36%


## Классификация

In [205]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def make_pipeline(classifier, n=2):
    """
    Создаёт pipeline для классификации имён 
    на основе переданного классификатора 
    и числа символов в n-граммах используемых в качестве признаков
    :param classifier: классификатор
    :param int: число символов в n-грамме
    """
    return Pipeline([
        ('vect', CountVectorizer(lowercase=False, analyzer='char', ngram_range=(n, n))),
        ('normalizer', StandardScaler(with_mean=False)),
        ('clf', classifier)
    ])

In [206]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def predict_and_score(clf, X_train, X_test, y_train, y_test):
    """Обучает, предсказывает и измеряет предсказания переданным классификатором на указанных данных"""
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    print("Accuracy:    {0:.2f}".format(accuracy_score(y_test, y_predict)))  
    print("F1-measure:  {0:.2f}".format(f1_score(y_test, y_predict, average='macro')))
    print("Precision:   {0:.2f}".format(precision_score(y_test, y_predict, average='macro')))
    print("Recall:      {0:.2f}".format(recall_score(y_test, y_predict, average='macro')))    
    return y_predict

In [207]:
from sklearn.linear_model import LogisticRegression

for n in [2, 3, 4]:
    print("\nРазбиение на n-граммы: n={}".format(n))
    clf = make_pipeline(LogisticRegression(random_state=random_state), n=n)
    predict_and_score(clf, X_train, X_test, y_train, y_test);


Разбиение на n-граммы: n=2
Accuracy:    0.86
F1-measure:  0.85
Precision:   0.85
Recall:      0.84

Разбиение на n-граммы: n=3
Accuracy:    0.83
F1-measure:  0.82
Precision:   0.82
Recall:      0.81

Разбиение на n-граммы: n=4
Accuracy:    0.81
F1-measure:  0.80
Precision:   0.80
Recall:      0.82


## Выводы
- Лучший результат при разбиении на 2-х символьные n-граммы как по accuracy, так и по F1

## Пытаемся улучшить модель

In [208]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

def tune_model(clf, parameters, names, cv=2, scoring='accuracy'):
    X, y = names.index, names.gender.cat.codes
    cv = StratifiedKFold(n_splits=cv, shuffle=True, random_state=random_state)
    grid_search = GridSearchCV(clf, parameters, cv=cv, scoring=scoring, n_jobs=1)
    grid_search.fit(X, y)

    print("Лучший результат: %0.3f" % grid_search.best_score_)
    print("Лучшие параметры:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
    print("\nНа тестовых данных:" % grid_search.best_score_)
    predict_and_score(grid_search.best_estimator_, *split_names_to_train_test(names));

In [209]:
tune_model(
    make_pipeline(LogisticRegression(random_state=random_state), n=2),
    {
        'clf__penalty': ['l1', 'l2'],
        'clf__C': [1, 2],
        'clf__tol': [0.0001, 0.00001, 0.000001],
    },
    names
)

Лучший результат: 0.845
Лучшие параметры:
	clf__C: 1
	clf__penalty: 'l1'
	clf__tol: 0.0001

На тестовых данных:
Размер обучающей выбрки: 5409
Размер тестовой выбрки: 1804
Процент мужчин в обучающей выборке: 36%
Процент мужчин в тестовой выборке: 36%
Accuracy:    0.86
F1-measure:  0.85
Precision:   0.86
Recall:      0.84


In [210]:
# Повторяем с penalty=l1
for n in [2, 3, 4]:
    print("\nРазбиение на n-граммы: n={}".format(n))
    clf = make_pipeline(LogisticRegression(random_state=random_state, penalty='l1'), n=n)
    predict_and_score(clf, X_train, X_test, y_train, y_test);


Разбиение на n-граммы: n=2
Accuracy:    0.86
F1-measure:  0.85
Precision:   0.86
Recall:      0.84

Разбиение на n-граммы: n=3
Accuracy:    0.83
F1-measure:  0.81
Precision:   0.83
Recall:      0.81

Разбиение на n-граммы: n=4
Accuracy:    0.79
F1-measure:  0.78
Precision:   0.78
Recall:      0.81


## Выводы
